In [ ]:
import os
os.environ['HF_HOME'] = '/expanse/lustre/projects/csd819/a3murali/cache/'

from huggingface_hub import login
login(token = "<hf_token>")

import transformers
import torch

import csv
import numpy as np
import pandas as pd

print(f"PyTorch Version: {torch.__version__}")
print(f"Transformers Version: {transformers.__version__}")

from transformers import AutoConfig, AutoModel, AutoTokenizer, AutoModelForCausalLM, pipeline, LlamaForCausalLM

model = "ivnle/llamatales_jr_8b-lay8-hs512-hd8-33M"
tokenizer = AutoTokenizer.from_pretrained(model)


llamatales_pipeline = pipeline("text-generation", model = model, device = "cuda")

# prompts = {
#     1: "Once upon a time there was a dragon", 
#     2: "Once upon a time there was a princess", 
#     3: "Once upon a time there were two children",
#     4: "Once upon a time there was a prince",
#     5: "Once upon a time there was a frog",
#     6: "Once upon a time there was a king",
#     7: "Once upon a time there was a queen",
#     8: "Once upon a time there was a wolf",
#     9: "Once upon a time there was a genie",
#     10: "Once upon a time there was a poor boy"
# }
prompts = {1: "Once upon a time there was a dragon"}

data = []
for prompt_id in prompts:
    #generation - generate stories for each prompt
    for i in range(2):
        sequences = llamatales_pipeline(
            prompts[prompt_id],
            do_sample = True,
            top_k = 10,
            num_return_sequences = 1,
            max_new_tokens = 512,
        )

        print(llamatales_pipeline.model.config)
        print(llamatales_pipeline.model.generation_config.to_dict())
        
        generated_story = sequences[0]['generated_text']
        print(generated_story)

        num_tokens_generated_story = len(tokenizer.encode(generated_story))

        #test - collect hidden states
        
        config = llamatales_pipeline.model.config #AutoConfig.from_pretrained("/expanse/lustre/projects/csd819/a3murali/cache/models--ivnle--llamatales_jr_8b-lay8-hs512-hd8-33M/snapshots/461f50f0024efb46b94dc68cc850d12d75ecb325", output_hidden_states = True)
        model = AutoModelForCausalLM.from_config(config).to('cuda')
        # if(i == 0):
        #     print(model)
            
        #     for j in model.named_parameters():
        #         print(f"{j[0]} -> {j[1].device}")


        tokenizer = AutoTokenizer.from_pretrained("/expanse/lustre/projects/csd819/a3murali/cache/models--ivnle--llamatales_jr_8b-lay8-hs512-hd8-33M/snapshots/461f50f0024efb46b94dc68cc850d12d75ecb325/", config = config)

        inputs = tokenizer(generated_story, return_tensors="pt").to("cuda")

        print("Input Shape: ", inputs.input_ids.shape[1])

        outputs = model.generate(inputs.input_ids, attention_mask=inputs["attention_mask"], do_sample = True, top_k = 10, num_return_sequences = 1, max_new_tokens = 100, eos_token_id = tokenizer.eos_token_id, pad_token_id = tokenizer.pad_token_id, output_hidden_states = True, return_dict_in_generate=True)

        print(tokenizer.batch_decode(outputs[0], skip_special_tokens=True, clean_up_tokenization_spaces=True)[0])
        #analysis - collect prompt id of each token
        output_id = []
        for token in range(num_tokens_generated_story):
            output_id.append(prompt_id)
        
        convert_hidden_states = []
        print("Hidden States Type: ", type(outputs.hidden_states))
        print("Hidden States Length: ", len(outputs.hidden_states))
        print("Hidden States Shape: ", outputs.sequences.shape)

        for i in outputs.hidden_states:
            convert_hidden_states.append([j.detach().cpu().numpy() for j in i])
        
        print(len(convert_hidden_states))
        print(len(convert_hidden_states[0]))
        print(len(convert_hidden_states[0][0]))
        print(len(convert_hidden_states[0][0][0]))
        print(len(convert_hidden_states[0][0][0][0]))
        # print(len(convert_hidden_states[0][0][0][0][0]))
        data.append([prompts[prompt_id], generated_story, convert_hidden_states, np.array(output_id)])

        #(([]))

df = pd.DataFrame(data, columns = ["prompt", "story", "hidden_states", "output_token_prompt_id"], dtype = 'object')
df

PyTorch Version: 2.7.0+cu126
Transformers Version: 4.52.4


Device set to use cuda


LlamaConfig {
  "architectures": [
    "LlamaForCausalLM"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "bos_token_id": 128000,
  "eos_token_id": 128009,
  "head_dim": 64,
  "hidden_act": "silu",
  "hidden_size": 512,
  "initializer_range": 0.02,
  "intermediate_size": 2048,
  "max_position_embeddings": 512,
  "mlp_bias": false,
  "model_type": "llama",
  "num_attention_heads": 8,
  "num_hidden_layers": 8,
  "num_key_value_heads": 8,
  "pretraining_tp": 1,
  "rms_norm_eps": 1e-05,
  "rope_scaling": null,
  "rope_theta": 500000.0,
  "tie_word_embeddings": false,
  "torch_dtype": "float32",
  "transformers_version": "4.52.4",
  "use_cache": true,
  "vocab_size": 128256
}

{'max_length': 20, 'max_new_tokens': None, 'min_length': 0, 'min_new_tokens': None, 'early_stopping': False, 'max_time': None, 'stop_strings': None, 'do_sample': False, 'num_beams': 1, 'num_beam_groups': 1, 'penalty_alpha': None, 'dola_layers': None, 'use_cache': True, 'cache_implementation': None, 'cach

In [2]:
# df.to_csv("story_dataset.csv", index = False)